In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# DRIVE_PATH = '/content/gdrive/MyDrive/RL-proj'
DRIVE_PATH = '/content/gdrive/MyDrive/cs182/RL-proj'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
# SYM_PATH = '/content/cs182/RL-proj'
# if not os.path.exists(SYM_PATH):
#   !ln -s $DRIVE_PATH $SYM_PATH

Mounted at /content/gdrive


In [3]:
%tensorflow_version 1.x
!pip install procgen -q
!pip install git+https://github.com/openai/baselines #> ~/pip_install_baselines.log
!pip install mpi4py -q

TensorFlow 1.x selected.
     |████████████████████████████████| 39.9MB 104kB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 3.3MB 37.6MB/s 
     |████████████████████████████████| 22.2MB 1.5MB/s 
     |████████████████████████████████| 675kB 39.7MB/s 
     |████████████████████████████████| 204kB 60.4MB/s 
     |████████████████████████████████| 40kB 3.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Cloning https://github.com/openai/baselines to /tmp/pip-req-build-ads3_ndx
  Running command git clone -q https://github.com/openai/baselines /tmp/pip-req-build-ads3_ndx
     |████████████████████████████████| 1.6MB 14.3MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp37-none-any.whl size=220664 sha256=78fdbf7a574d41b368ab6b69abf93d41ef0e942c1d4d96e2dd3368c2dfad604b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rmiekifh/wheels/d8/55/79/5387046

In [4]:
import time, os
import tensorflow as tf
from baselines.ppo2 import ppo2
from baselines.common.models import build_impala_cnn
from baselines.common.mpi_util import setup_mpi_gpus
from procgen import ProcgenEnv
from baselines.common.vec_env import (
    VecExtractDictObs,
    VecMonitor,
    VecFrameStack,
    VecNormalize
)
from baselines.bench import Monitor
from baselines import logger
from mpi4py import MPI
import gym

In [27]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

In [ ]:
os.listdir(DRIVE_PATH+'/log') 
# os.listdir('/content/gdrive/MyDrive')

['PPO_CNN_10lvl',
 'PPO_10lvl',
 'PPO_50lvl',
 'ACER_50lvl',
 'baselines',
 'good_PPO_50lvl']

In [7]:
timestamp = "110521_23:08:32"
save_dir = DRIVE_PATH + "/log/baselines/" + timestamp 

last_ckpt = os.listdir(save_dir + "/checkpoints") [-1]

# ckpt = save_dir + "/checkpoints/00001"
ckpt = save_dir + "/checkpoints/" + last_ckpt

In [8]:
log_dir = '/tmp/'
log_dir = os.path.join(log_dir, time.strftime("%d%m%y_%H:%M:%S", time.localtime()))
os.makedirs(log_dir, exist_ok=True)

logger.configure(dir=log_dir)

# tf.reset_default_graph()

num_levels = 50
venv = ProcgenEnv(num_envs=1, env_name="fruitbot", num_levels=num_levels, distribution_mode="easy")
venv = VecExtractDictObs(venv, "rgb")
venv = VecMonitor(
    venv=venv, filename=None, keep_buf=100,
)
venv = VecNormalize(venv=venv, ob=False)


conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)


total_timesteps = 0#int(2e6)

learning_rate = 5e-4
ent_coef = .01
gamma = .999
lam = .95
nsteps = 256
nminibatches = 8
ppo_epochs = 3
clip_range = .2
use_vf_clipping = None
# comm = MPI.COMM_WORLD
# rank = comm.Get_rank()
model = ppo2.learn(
    env=venv,
    network=conv_fn,
    total_timesteps=total_timesteps,
    # save_interval=0,
    nsteps=nsteps,
    nminibatches=nminibatches,
    lam=lam,
    gamma=gamma,
    noptepochs=ppo_epochs,
    log_interval=1,
    save_interval=100,
    ent_coef=ent_coef,
    # mpi_rank_weight=mpi_rank_weight,
    clip_vf=use_vf_clipping,
    # comm=comm,
    lr=learning_rate,
    cliprange=clip_range,
    update_fn=None,
    init_fn=None,
    vf_coef=0.5,
    max_grad_norm=0.5,
    load_path=ckpt
)

Logging to /tmp/120521_00:57:44








Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [9]:
model

In [17]:
import numpy as np

def evaluate(model, num_episodes=100, num_levels=50, unseen=True):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    start_level = 0
    if unseen:
        start_level += num_levels
    venv = ProcgenEnv(num_envs=1, env_name="fruitbot", num_levels=num_levels, start_level=start_level, distribution_mode="easy")
    venv = VecExtractDictObs(venv, "rgb")
    venv = VecMonitor(
        venv=venv, filename=None, keep_buf=100,
    )
    venv = VecNormalize(venv=venv, ob=False)
    env = venv
    # env = model.get_env()

    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        dones = False
        obs = env.reset()
        states = model.initial_state
        while not dones:
            actions, values, states, neglogpacs = model.step(obs, S=states, M=dones)

            # action, _states = model.predict(obs)

            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, rewards, dones, infos = env.step(actions)
            episode_rewards.append(rewards)
        print(i, [r[0] for r in episode_rewards])
        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [18]:
evaluate(model)

0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.544397, 0.0, 0.0, 0.0, 0.0, -9.361819, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

-1.0651417